In [29]:
import pandas as pd 
author_url ='Astrid+Lindgren'
URL = f"http://libris.kb.se/xsearch?query={author_url}&format=json&n=50"
df = pd.read_json(URL)
df = df['xsearch']
df = df['list']
df = pd.json_normalize(df)
df_out = df.loc[(df.type=='book')&(df.language=='swe')].reset_index()
df_libris = df_out.to_dict()

In [27]:
author = author_url.split('+')
auth_libris = df_libris['creator'][0]
if author[1] in auth_libris and author[0] in auth_libris:
    print(auth_libris)


In [28]:
import pandas as pd
import requests
import json
author_url ='George+Orwell'

url_OL1 = 'https://openlibrary.org/search/authors.json?q='+f'{author_url}'
response = requests.get(url_OL1)
url_OLD1 = response.json()
url_OLD1 = url_OLD1['docs']
df_OL = pd.json_normalize(url_OLD1)
df_OL.head()
#df_OL_out = df_OL.loc[(df_OL.key)&(df_OL.name)& (df_OL.alternate_names)&(df_OL.top_work)&(df_OL.top_subjects)].reset_index()
df_OL_out = df_OL.loc[0]
df_OL_out
OL = df_OL_out['key']
top_work = df_OL['top_work']
df_OL1 = df_OL_out.to_dict()

In [31]:
url_OL2 = f'https://openlibrary.org/authors/{OL}.json'
response = requests.get(url_OL2)
url_OLD2 = response.json()
df_OL2 = pd.json_normalize(url_OLD2)
try:
    bio_OL = df_OL2.loc[0]['bio.value']
except:
    bio_OL = 'No Bio Exist'
viaf = df_OL2.loc[0]['remote_ids.viaf']
isni = df_OL2.loc[0]['remote_ids.isni']
bio_OL, isni, viaf

('No Bio Exist', '0000000121441305', '95155403')

In [21]:
url_OL3 = f'https://openlibrary.org/authors/{OL}/works.json'
response = requests.get(url_OL3)
url_OLD3 = response.json()
df_OL3 = pd.json_normalize(url_OLD3, record_path =['entries'])
df_OL3['works'] = list(df_OL3.index)
df_OL3['main_auth'] = list(df_OL3.index)
df_OL3['auth'] = list(df_OL3.index)

def replace_key(x):
    work = df_OL3.loc[x]['key'].replace('/works/','')
    return work

def check_auth(x):
    main = df_OL3['authors'][x][0]['author']['key']
    return main

def replace_auth_key(x):
    auth = df_OL3.loc[x]['main_auth'].replace('/authors/','')
    return auth

"""    try:
        if df_OL3['authors'][x][0]['author']['key']:
            main = df_OL3['authors'][x][0]['author']['key']
            return main
    except:
        not_main = df_OL3['authors'][x][0]['type']['key']
        return not_main
"""

df_OL3['works'] = df_OL3["works"].apply(lambda x=df_OL3.index: replace_key(x))
df_OL3['main_auth'] = df_OL3["main_auth"].apply(lambda x=df_OL3.index: check_auth(x))
df_OL3['auth'] = df_OL3["auth"].apply(lambda x=df_OL3.index: replace_auth_key(x))
df_OL3_out = df_OL3[["works","auth","title","description","subjects"]]
df_OL3_Dict = df_OL3_out.to_dict()

In [22]:
url_OL3 = f'https://openlibrary.org/authors/{OL}/works.json'
response = requests.get(url_OL3)
url_OLD3 = response.json()
df_OL3 = pd.json_normalize(url_OLD3, record_path =['entries'])
df_OL3['works'] = list(df_OL3.index)
df_OL3['main_auth'] = list(df_OL3.index)
df_OL3['auth'] = list(df_OL3.index)

def replace_key(x):
    work = df_OL3.loc[x]['key'].replace('/works/','')
    return work

def check_auth(x):
    main = df_OL3['authors'][x][0]['author']['key']
    return main

def replace_auth_key(x):
    auth = df_OL3.loc[x]['main_auth'].replace('/authors/','')
    return auth

"""    try:
        if df_OL3['authors'][x][0]['author']['key']:
            main = df_OL3['authors'][x][0]['author']['key']
            return main
    except:
        not_main = df_OL3['authors'][x][0]['type']['key']
        return not_main
"""
df_OL3['works'] = df_OL3["works"].apply(lambda x=df_OL3.index: replace_key(x))
df_OL3['main_auth'] = df_OL3["main_auth"].apply(lambda x=df_OL3.index: check_auth(x))
df_OL3['auth'] = df_OL3["auth"].apply(lambda x=df_OL3.index: replace_auth_key(x))
df_OL3_out = df_OL3[["works","auth","title","description","subjects"]]
df_OL3_Dict = df_OL3_out.to_dict()

In [23]:
def get_book(x):
    #url_OL4 = f"https://openlibrary.org/works/{x}.json"
    url_OL4 = f"https://openlibrary.org/works/{x}/editions.json"
    response = requests.get(url_OL4)
    url_OLD4 = response.json()
    df_OLD4 = pd.json_normalize(url_OLD4, record_path =['entries'])
    df_OLD4['book_id'] = df_OLD4['key'].str.replace('/books/', '')
    return df_OLD4

In [24]:
def isbn_check(df_OL3_out):
    isbn_dict = {}
    for i in df_OL3_out.index:
        try:
            df_OLD4 = get_book(df_OL3_out.loc[i]['works'])
            isbn_dict[df_OL3_out.loc[i]['works']] = {}
            isbn_dict[df_OL3_out.loc[i]['works']]['book_id'] = df_OLD4['book_id'][0]
            isbn_dict[df_OL3_out.loc[i]['works']]['title'] = df_OLD4['title'][0]
            isbn_dict[df_OL3_out.loc[i]['works']]['isbn_10'] = 'none'
            isbn_dict[df_OL3_out.loc[i]['works']]['isbn_13'] = 'none'
            if isinstance(df_OLD4["book_id"], list):
                isbn_dict[df_OL3_out.loc[i]['works']]['book_id'] = df_OLD4['book_id'][0]
            else:
                isbn_dict[df_OL3_out.loc[i]['works']]['book_id'] = df_OLD4['book_id'][0]
            if 'isbn_10' in df_OLD4:
                if isinstance(df_OLD4["isbn_10"], list):
                    isbn_dict[df_OL3_out.loc[i]['works']]['isbn_10'] = df_OLD4['isbn_10'][0]
                else:
                    isbn_dict[df_OL3_out.loc[i]['works']]['isbn_10'] = df_OLD4['isbn_10'][0]
            if 'isbn_13' in df_OLD4:
                if isinstance(df_OLD4["isbn_13"], list):
                    isbn_dict[df_OL3_out.loc[i]['works']]['isbn_13'] = df_OLD4['isbn_13'][0]
                else:
                    isbn_dict[df_OL3_out.loc[i]['works']]['isbn_13'] = df_OLD4['isbn_13'][0]
        except:
            print('multiple editions of same book exists')
    return isbn_dict


In [26]:
isbn_dict = isbn_check(df_OL3_out)
isbn= pd.DataFrame.from_dict(isbn_dict,orient='index')
isbn.head()

multiple editions of same book exists


,book_id,title,isbn_10,isbn_13
OL18183435W,OL2224770M,The Winchester Reader,[0312048807],none
OL24500055W,OL32464325M,Statesmanship,[1474619274],[9781474619271]
OL18165436W,OL10077458M,Prentice Hall Literature,[0130624306],[9780130624307]
OL24558288W,OL7343807M,Prentice Hall: Literature,[0137224559],[9780137224555]
OL9319238W,OL42059292M,Prentice Hall Literature,[013131758X],[9780131317581]
